In [1]:
# Imports iniciais
import pandas as pd
import numpy as np

### Leitura e entendimento das bases

Nesta etapa, realizamos a leitura das quatro bases fornecidas no desafio:

- **base_cadastral.csv**: informações fixas dos clientes (porte, segmento, localização, etc.);
- **base_info.csv**: dados mensais de acompanhamento (renda e número de funcionários);
- **base_pagamentos_desenvolvimento.csv**: histórico de cobranças já pagas, usada para construir a variável-alvo (`INADIMPLENTE`);
- **base_pagamentos_teste.csv**: cobranças recentes ainda não pagas, usada posteriormente para gerar as previsões finais.

As bases são lidas a partir da pasta `data/` e posteriormente integradas em uma base única de modelagem.

In [2]:
# Caminho para a pasta dos dados
data_path = "./data/"

# leitura das bases
base_cadastral = pd.read_csv(data_path + 'base_cadastral.csv', sep=';')
base_info = pd.read_csv(data_path + 'base_info.csv', sep=';')
base_dev = pd.read_csv(data_path + 'base_pagamentos_desenvolvimento.csv', sep=';')
base_teste = pd.read_csv(data_path + 'base_pagamentos_teste.csv', sep=';')

### Junção das bases

Nesta etapa, unificamos as três bases (`pagamentos`, `info`, `cadastral`) para formar a base de modelagem.
Usamos `how="left"` para garantir que todos os registros de pagamento sejam mantidos.

In [3]:
# Criar base para modelagem ("merge")
df_modelagem = (
            base_dev
            .merge(base_info, on=["ID_CLIENTE", "SAFRA_REF"], how="left")
            .merge(base_cadastral, on="ID_CLIENTE", how="left")
)

print(f'Shape da base de modelagem: {df_modelagem.shape}')

Shape da base de modelagem: (77414, 16)


In [4]:
df_modelagem.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65


In [5]:
df_modelagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              77414 non-null  int64  
 1   SAFRA_REF               77414 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object 
 3   DATA_PAGAMENTO          77414 non-null  object 
 4   DATA_VENCIMENTO         77414 non-null  object 
 5   VALOR_A_PAGAR           76244 non-null  float64
 6   TAXA                    77414 non-null  float64
 7   RENDA_MES_ANTERIOR      71282 non-null  float64
 8   NO_FUNCIONARIOS         69827 non-null  float64
 9   DATA_CADASTRO           77414 non-null  object 
 10  DDD                     70000 non-null  object 
 11  FLAG_PF                 219 non-null    object 
 12  SEGMENTO_INDUSTRIAL     75997 non-null  object 
 13  DOMINIO_EMAIL           76516 non-null  object 
 14  PORTE                   74938 non-null

## Limpeza e padronização dos dados

Antes de seguir para a criação da variável-alvo, é importante realizar uma **limpeza estrutural** dos dados, garantindo que as informações estejam consistentes e em formato adequado.

Essa etapa tem dois objetivos principais:
1. **Garantir integridade** — assegurar que os dados de data e identificação estejam corretos para o cálculo da variável `INADIMPLENTE`.
2. **Remover ou ajustar inconsistências estruturais** — como colunas irrelevantes, duplicatas e tipos incorretos.

### Principais ações nesta etapa:

- **Conversão de colunas de data** para o tipo `datetime`, evitando erros no cálculo de atraso.
- **Remoção de colunas com pouca informação**, como `FLAG_PF`, que possui quase todos os valores nulos.
- **Eliminação de duplicatas** baseadas em `ID_CLIENTE` e `SAFRA_REF`.
- **Padronização de tipos** (como `CEP_2_DIG` para string).
- **Verificação de nulos críticos** — especialmente em colunas de datas de vencimento e pagamento.

Após essa limpeza estrutural, os dados estarão prontos para a criação da variável-alvo e o início do tratamento de valores faltantes e inconsistências numéricas/categóricas.

In [6]:
# Gerar cópia de segurança
df_modelagem_backup = df_modelagem.copy()

# Cópia de trabalho
df_modelagem = df_modelagem.copy()

In [7]:
# Conversão de colunas de datas
for col in ['DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO', 'DATA_PAGAMENTO', 'DATA_CADASTRO']:
    df_modelagem[col] = pd.to_datetime(df_modelagem[col], errors='coerce')

# Testar se a conversão funcionou (deve retornar "dtype('<M8[ns]')")
df_modelagem['DATA_VENCIMENTO'].dtype

dtype('<M8[ns]')

In [8]:
df_modelagem.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,NaN,NaN,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65


In [9]:
# Diagnóstico inicial de valores ausentes
nulls = df_modelagem.isnull().mean().sort_values(ascending=False)
nulls.head(12)

FLAG_PF                   0.997171
NO_FUNCIONARIOS           0.098006
DDD                       0.095771
RENDA_MES_ANTERIOR        0.079210
PORTE                     0.031984
SEGMENTO_INDUSTRIAL       0.018304
VALOR_A_PAGAR             0.015114
DOMINIO_EMAIL             0.011600
TAXA                      0.000000
DATA_VENCIMENTO           0.000000
DATA_EMISSAO_DOCUMENTO    0.000000
DATA_PAGAMENTO            0.000000
dtype: float64

In [10]:
# Remover FLAG_PF (sem informações relevantes)
df_modelagem = df_modelagem.drop(columns=['FLAG_PF']) 

type(df_modelagem)

pandas.core.frame.DataFrame

In [11]:
# Criar flags de ausência de informação

# Colunas numéricas e categóricas com nulos
num_cols = ['VALOR_A_PAGAR', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']
cat_cols = ['SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'DDD']

# Para cada coluna com possíveis valores ausentes, criamos uma flag binária (0 ou 1)
# A ideia é preservar a informação de "ausência", que pode ter valor preditivo.
# Exemplo: um cliente que não informou a renda pode ter comportamento de pagamento diferente.
# Isso evita apagar o sinal da ausência quando fizermos imputações (fillna).

for col in num_cols + cat_cols:
    if col in df_modelagem.columns:
        df_modelagem[f'FLAG_MISSING__{col}'] = df_modelagem[col].isna().astype(int)

In [12]:
# Imputar valores nulos

# Colunas numéricas: preencher com a mediana
# A mediana é menos sensível a outliers do que a média.
# Com os flags de ausência criados acima, preservamos a informação de que o valor original era nulo.
num_cols = ['VALOR_A_PAGAR', 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']

for col in num_cols:
    if col in df_modelagem.columns:
        median_value = df_modelagem[col].median()
        df_modelagem[col] = df_modelagem[col].fillna(median_value)

In [ ]:
# Diagnóstico do que segue nulo
df_modelagem.isnull().sum().sort_values(ascending=False)

DDD                                  7414
PORTE                                2476
SEGMENTO_INDUSTRIAL                  1417
DOMINIO_EMAIL                         898
ID_CLIENTE                              0
SAFRA_REF                               0
VALOR_A_PAGAR                           0
DATA_VENCIMENTO                         0
DATA_PAGAMENTO                          0
DATA_EMISSAO_DOCUMENTO                  0
DATA_CADASTRO                           0
NO_FUNCIONARIOS                         0
RENDA_MES_ANTERIOR                      0
TAXA                                    0
CEP_2_DIG                               0
FLAG_MISSING__VALOR_A_PAGAR             0
FLAG_MISSING__RENDA_MES_ANTERIOR        0
FLAG_MISSING__NO_FUNCIONARIOS           0
FLAG_MISSING__SEGMENTO_INDUSTRIAL       0
FLAG_MISSING__DOMINIO_EMAIL             0
FLAG_MISSING__PORTE                     0
FLAG_MISSING__DDD                       0
dtype: int64

In [16]:
# Inputar variáveis categóricas restantes

# DDD -> preencher com '00'
df_modelagem['DDD'] = (
    df_modelagem['DDD']
    .astype(str)
    .str.extract(r'(\d+)')[0]  # extrai só números
    .fillna('00')
    .str.zfill(2)
)


# Demais variáveis categóricas → preenche com 'Desconhecido'
for col in ['PORTE', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL']:
    df_modelagem[col] = df_modelagem[col].fillna('Desconhecido').astype(str)

In [17]:
# Diagnóstico final de valores ausentes
df_modelagem.isnull().sum().sort_values(ascending=False)

ID_CLIENTE                           0
SAFRA_REF                            0
DATA_EMISSAO_DOCUMENTO               0
DATA_PAGAMENTO                       0
DATA_VENCIMENTO                      0
VALOR_A_PAGAR                        0
TAXA                                 0
RENDA_MES_ANTERIOR                   0
NO_FUNCIONARIOS                      0
DATA_CADASTRO                        0
DDD                                  0
SEGMENTO_INDUSTRIAL                  0
DOMINIO_EMAIL                        0
PORTE                                0
CEP_2_DIG                            0
FLAG_MISSING__VALOR_A_PAGAR          0
FLAG_MISSING__RENDA_MES_ANTERIOR     0
FLAG_MISSING__NO_FUNCIONARIOS        0
FLAG_MISSING__SEGMENTO_INDUSTRIAL    0
FLAG_MISSING__DOMINIO_EMAIL          0
FLAG_MISSING__PORTE                  0
FLAG_MISSING__DDD                    0
dtype: int64